In [1]:
!cd

C:\Users\SEC\OneDrive\바탕 화면\2021-K-Digital-Training-main\SQL


## 결측치 제거

In [2]:
# 새출발
import pandas as pd
import pymysql.cursors
import numpy as np

In [ ]:
sql="select customers.customerName, payments.checkNumber from customers left join payments on customers.customerNumber=payments.customerNumber where payments.paymentDate >= '2004-10-06';"


In [3]:
# MySQL DB에서 데이터 받아와서 DataFrame에 저장

conn = pymysql.connect(host='localhost', user='root', 
                       password='054867', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)

In [4]:
try:

   with conn.cursor() as curs:
      sql="SELECT * FROM tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)
      print(df)

finally:

   conn.close()

     total_bill   tip     sex smoker   day    time  size
0         16.99  1.01  Female     No   Sun  Dinner     2
1         10.34  1.66    Male     No   Sun  Dinner     3
2         21.01  3.50    Male     No   Sun  Dinner     3
3         23.68  3.31    Male     No   Sun  Dinner     2
4         24.59  3.61  Female     No   Sun  Dinner     4
..          ...   ...     ...    ...   ...     ...   ...
243       18.78  3.00  Female     No  Thur  Dinner     2
244       16.99  1.01  Female     No   Sun  Dinner     2
245       26.88  3.12    Male     No        Dinner     4
246       26.88  3.12    Male     No        Dinner     4
247       26.88  3.12    Male     No        Dinner     5

[248 rows x 7 columns]


In [5]:
df['tip_rate']=df.tip/df.total_bill*100

In [6]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,Female,No,Sun,Dinner,2,5.944673
1,10.34,1.66,Male,No,Sun,Dinner,3,16.054159
2,21.01,3.50,Male,No,Sun,Dinner,3,16.658734
3,23.68,3.31,Male,No,Sun,Dinner,2,13.978041
4,24.59,3.61,Female,No,Sun,Dinner,4,14.680765


In [7]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
tip_rate      0
dtype: int64

In [8]:
df.replace('',np.nan,inplace=True) 

In [9]:
df.sex.replace({'Female':0,'Male':1},inplace=True)

In [10]:
df.smoker.replace({'No':0,'Yes':1},inplace=True)
df.day.replace({'Thur':0,'Fri':1,'Sat':2,'Sun':3},inplace=True)
df.time.replace({'Lunch':0,'Dinner':1},inplace=True)

In [11]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,0,0,3.0,1,2,5.944673
1,10.34,1.66,1,0,3.0,1,3,16.054159
2,21.01,3.50,1,0,3.0,1,3,16.658734
3,23.68,3.31,1,0,3.0,1,2,13.978041
4,24.59,3.61,0,0,3.0,1,4,14.680765


In [12]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           3
time          0
size          0
tip_rate      0
dtype: int64

In [ ]:
type(df)

In [13]:
# from numpy import isnan
from sklearn.impute import SimpleImputer

In [14]:
data = df.values

In [15]:
df.describe()

,total_bill,tip,sex,smoker,day,time,size,tip_rate
count,248.000000,248.000000,248.000000,248.000000,245.000000,248.000000,248.000000,248.000000
mean,19.860484,2.991734,0.645161,0.375000,1.730612,0.725806,2.588710,15.985279
std,8.866048,1.378257,0.479432,0.485102,1.156274,0.447009,0.965075,6.110974
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.563814
25%,13.385000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,12.534046
50%,17.815000,2.960000,1.000000,0.000000,2.000000,1.000000,2.000000,15.384615
75%,24.527500,3.520000,1.000000,1.000000,3.000000,1.000000,3.000000,19.065829
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000,71.034483


In [16]:
# flatten() 여러 칼럼을 하나로 합친다. 
sum(np.isnan(data).flatten())

3

In [17]:
imputer = SimpleImputer(strategy='median')

In [18]:
# fit on the dataset
imputer.fit(data)

SimpleImputer(strategy='median')

In [19]:
# transform the dataset
data_trans = imputer.transform(data)

In [20]:
df_trans=pd.DataFrame(data_trans)

In [21]:
df_trans.isnull().sum()

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
dtype: int64

## RFE 적용

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  248 non-null    float64
 1   tip         248 non-null    float64
 2   sex         248 non-null    int64  
 3   smoker      248 non-null    int64  
 4   day         245 non-null    float64
 5   time        248 non-null    int64  
 6   size        248 non-null    int64  
 7   tip_rate    248 non-null    float64
dtypes: float64(4), int64(4)
memory usage: 15.6 KB


In [26]:
y=df.tip

In [27]:
X=df.drop('tip',axis=1)

In [28]:
X

,total_bill,sex,smoker,day,time,size,tip_rate
0,16.99,0,0,3.0,1,2,5.944673
1,10.34,1,0,3.0,1,3,16.054159
2,21.01,1,0,3.0,1,3,16.658734
3,23.68,1,0,3.0,1,2,13.978041
4,24.59,0,0,3.0,1,4,14.680765
...,...,...,...,...,...,...,...
243,18.78,0,0,0.0,1,2,15.974441
244,16.99,0,0,3.0,1,2,5.944673
245,26.88,1,0,NaN,1,4,11.607143
246,26.88,1,0,NaN,1,4,11.607143


In [29]:
y.describe()

count    248.000000
mean       2.991734
std        1.378257
min        1.000000
25%        2.000000
50%        2.960000
75%        3.520000
max       10.000000
Name: tip, dtype: float64

In [30]:
data=X.values

In [31]:
y=y.values

In [32]:
sum(np.isnan(data).flatten())

3

In [33]:
sum(np.isnan(y).flatten())

0

In [34]:
imputer = SimpleImputer(strategy='median')

In [35]:
imputer.fit(data)

SimpleImputer(strategy='median')

In [36]:
data_trans=imputer.transform(data)

In [37]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR

In [38]:
estimator=SVR(kernel='linear')

In [39]:
rfe = RFE(estimator, n_features_to_select=4)

In [40]:
data_trans.shape

(248, 7)

In [41]:
rfe.fit(data_trans,y)

RFE(estimator=SVR(kernel='linear'), n_features_to_select=4)

In [42]:
for i in range(X.shape[1]):
  print('Column: %10s, Rank: %d' % (X.columns[i], rfe.ranking_[i]))

Column: total_bill, Rank: 1
Column:        sex, Rank: 2
Column:     smoker, Rank: 1
Column:        day, Rank: 4
Column:       time, Rank: 3
Column:       size, Rank: 1
Column:   tip_rate, Rank: 1


In [48]:
X.columns

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size', 'tip_rate'], dtype='object')

In [44]:
df1=pd.DataFrame(data_trans, columns=['total_bill', 'sex', 'smoker', 'day', 'time', 'size','tip_rate'])

In [45]:
df1.drop(['sex','day','time'],axis=1)

,total_bill,smoker,size,tip_rate
0,16.99,0.0,2.0,5.944673
1,10.34,0.0,3.0,16.054159
2,21.01,0.0,3.0,16.658734
3,23.68,0.0,2.0,13.978041
4,24.59,0.0,4.0,14.680765
...,...,...,...,...
243,18.78,0.0,2.0,15.974441
244,16.99,0.0,2.0,5.944673
245,26.88,0.0,4.0,11.607143
246,26.88,0.0,4.0,11.607143


## MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
trans=MinMaxScaler()

In [ ]:
X_norm=trans.fit_transform(data)

In [ ]:
df_norm=pd.DataFrame(X_norm)

In [ ]:
df_norm.describe()

## StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc=StandardScaler()

In [ ]:
df_sc = sc.fit_transform(data)

In [ ]:
df_transform_sc=pd.DataFrame(df_sc)

In [ ]:
df_transform_sc.describe().round()

In [ ]:
data_trans

## PCA

In [46]:
from sklearn.decomposition import PCA

In [47]:
trans = PCA(n_components=4)
data_dim = trans.fit_transform(data_trans)

In [48]:
trans.fit(data_dim)

PCA(n_components=4)

In [49]:
trans.explained_variance_ratio_

array([0.72596385, 0.25672895, 0.01217223, 0.00513497])

In [50]:
print(data_dim[:3,:])

[[ -0.85984111 -10.39293417  -1.42457464  -0.31313185]
 [  8.85427077  -3.24437819  -1.66295355   0.98804187]
 [ -0.88781948   1.07573919  -1.30012266   0.31700206]]


In [51]:
trans.components_.round()

array([[ 1.,  0.,  0.,  0.],
       [-0.,  1., -0., -0.],
       [-0., -0.,  1.,  0.],
       [ 0., -0., -0.,  1.]])

In [52]:
pca_x=pd.DataFrame(data_dim)

In [53]:
pca_x.describe().round(2)

,0,1,2,3
count,248.00,248.00,248.00,248.00
mean,0.00,0.00,0.00,-0.00
std,9.27,5.51,1.20,0.78
min,-28.63,-10.39,-1.66,-2.04
25%,-5.03,-3.43,-1.04,-0.45
50%,2.02,-0.61,-0.38,-0.15
75%,6.26,2.63,1.20,0.39
max,31.25,47.06,2.55,3.07


In [54]:
columns = ['pca_comp_%i' % i for i in range(4)]
df_pca  = pd.DataFrame(trans.fit(data_dim).transform(pca_x), columns=columns, index=pca_x.index)
df_pca.head()

,pca_comp_0,pca_comp_1,pca_comp_2,pca_comp_3
0,-0.859841,-10.392934,-1.424575,-0.313132
1,8.854271,-3.244378,-1.662954,0.988042
2,-0.887819,1.075739,-1.300123,0.317002
3,-4.267540,-0.524842,-1.182127,-0.796737
4,-4.980404,0.517564,-1.182651,1.081223


## 회귀분석

In [55]:
# pearson's correlation feature selection for numeric input and numeric output
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [56]:
# define feature selection
fs = SelectKBest(score_func=f_regression, k=4)

In [57]:
# apply feature selection
X_selected = fs.fit_transform(data_trans, y)

In [75]:
df_col=fs.get_support(indices=True)

In [76]:
x_col=X.columns[df_col]

In [77]:
print(X_selected.shape)

(248, 4)


In [78]:
pd.DataFrame(X_selected,columns=x_col)

,total_bill,day,size,tip_rate
0,16.99,3.0,2.0,5.944673
1,10.34,3.0,3.0,16.054159
2,21.01,3.0,3.0,16.658734
3,23.68,3.0,2.0,13.978041
4,24.59,3.0,4.0,14.680765
...,...,...,...,...
243,18.78,0.0,2.0,15.974441
244,16.99,3.0,2.0,5.944673
245,26.88,2.0,4.0,11.607143
246,26.88,2.0,4.0,11.607143
